In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

In [2]:
cifar_train=dset.CIFAR10("CIFAR10/",train=True,transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.4913997551666284, 0.48215855929893703, 0.4465309133731618), std=(0.24703225141799082, 0.24348516474564, 0.26158783926049628))
                                     ]),target_transform=None,download=True)
cifar_test=dset.CIFAR10("CIFAR10/",train=False,
                        transform=transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=(0.4913997551666284, 0.48215855929893703, 0.4465309133731618), std=(0.24703225141799082, 0.24348516474564, 0.26158783926049628))])
                                      ,target_transform=None,download=True)


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/
Files already downloaded and verified


In [3]:
cifar_train+=dset.CIFAR10("CIFAR10/",train=True,transform=transforms.Compose([transforms.Resize((32,32)),  #resises the image so it can be perfect for our model.
                                      transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
                                      transforms.RandomRotation(10),     #Rotates the image to a specified angel
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
                                      transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) #Normalize all the images
                               ]),target_transform=None,download=True)

Files already downloaded and verified


In [4]:
len(cifar_train)

100000

In [5]:
def ComputeAccr(dloader, imodel):
  correct = 0
  total = 0

  for j, [imgs, labels] in enumerate(dloader):
    img = Variable(imgs,volatile=True).cuda()
    label = Variable(labels).cuda()

    output = imodel.forward(img)
    _, output_index = torch.max(output,1)

    total += label.size(0)
    correct += (output_index == label).sum().float()
  print("Accuracy of Test Data: {}".format(100*correct/total))

In [6]:
import torch.nn.functional as F

def mish(x): 
  return x * torch.tanh(F.softplus(x))

In [7]:
class Mish(nn.Module):
    def __init__(self):
        super(Mish,self).__init__()

    def forward(self, input):
        return mish(input)

In [9]:
learning_rate = 0.005
batch_size = 16
num_epoch = 20

In [10]:
activation_function = Mish()

train_loader = torch.utils.data.DataLoader(list(cifar_train)[:],batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(cifar_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer=nn.Sequential(
        nn.Conv2d(3,16,3,padding=1),
        nn.BatchNorm2d(16),
        activation_function,
        nn.Dropout2d(0.1),

        nn.Conv2d(16,32,3,padding=1),
        nn.BatchNorm2d(32),
        activation_function,
        nn.Dropout2d(0.2),
        nn.MaxPool2d(2,2),
        
        nn.Conv2d(32,64,3,padding=1),
        nn.BatchNorm2d(64),
        activation_function,
        nn.Dropout2d(0.3),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer=nn.Sequential(
        nn.Linear(64*8*8,1024),
        nn.BatchNorm1d(1024),
        activation_function,

        nn.Linear(1024,512),
        nn.BatchNorm1d(512),
        activation_function,


        nn.Linear(512,256),
        nn.BatchNorm1d(256),
        activation_function,

        nn.Linear(256,128),
        nn.BatchNorm1d(128),
        activation_function,

        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        activation_function,

        nn.Linear(64,10),
        nn.BatchNorm1d(10),
        activation_function,
        
        nn.Dropout2d(0.5),
        nn.LogSoftmax(dim=1)
    )
    

  def forward(self,x):
    out=self.layer(x)
    out=out.view(batch_size,-1)
    out=self.fc_layer(out)
    
    return out

model = CNN().cuda()

In [ ]:
loss_func = nn.CrossEntropyLoss()
decay = learning_rate/num_epoch
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate,weight_decay=decay)
scheduler = lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

for i in range(200):
  print("epoch_cnt : ",i)
  for j, [image,label] in enumerate(train_loader):
    model.train()
    x=Variable(image).cuda()
    y_=Variable(label).cuda()

    optimizer.zero_grad()
    output=model.forward(x)
    loss=loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if j%1000==0:
      print(loss)

  model.eval()
  if i%10 == 0:
    ComputeAccr(train_loader,model)
    ComputeAccr(test_loader,model)
    print("\n")

epoch_cnt :  0
tensor(2.6477, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5125, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.1262, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2107, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.1782, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2095, device='cuda:0', grad_fn=<NllLossBackward>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 51.15299987792969
Accuracy of Test Data: 54.14999771118164


epoch_cnt :  1
tensor(1.9525, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8508, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.9699, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8391, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8732, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.1775, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0384, device='cuda:0', grad_fn=<NllLossBackward>)
epoch_cnt :  2
tensor(1.9642, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5869, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0556, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.7729, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5594, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5770, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8825, device='cuda:0', grad_fn=<NllLossBackward>)
epoch_cnt :  3
tensor(1.8472, device='cuda:0', grad_fn=<NllL

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
cd'/content/drive/MyDrive/Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [20]:
netname = 'kimjinsung_mpl_weight.pkl'
# torch.save(model,netname,)

model = torch.load(netname)


model.eval()
ComputeAccr(test_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 79.87999725341797
